# Model Comparison
---

1. Data Preparation
2. Model Comparison
- K-mean
- LDA
- NMF
- Top2Vec
- BERTopic > give up BERTopic because of time constraints

## 0. Import Libraries

In [2]:
# Import Basic Libraries
import pandas as pd
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns

# Import Sklearn Libraries
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix, plot_confusion_matrix
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.feature_extraction import DictVectorizer
from sklearn.decomposition import NMF
from sklearn.cluster import KMeans


# Import NLP Libraries
from gensim.models import LdaModel
from gensim.corpora.dictionary import Dictionary
from pythainlp.tokenize import sent_tokenize, word_tokenize
from pythainlp.corpus import thai_stopwords
import re
from tqdm import tqdm 
import pyLDAvis
import pyLDAvis.gensim_models
from top2vec import Top2Vec


# Set default Thai font
mpl.font_manager.fontManager.addfont('./THSarabunNew/THSarabunNew.ttf')
mpl.rc('font', family='TH Sarabun New', size=20)

# Suppress warnings
import warnings
warnings.filterwarnings('ignore')
warnings.

/Users/Anik/opt/anaconda3/lib/python3.9/site-packages/past/builtins/misc.py:45: DeprecationWarning: the imp module is deprecated in favour of importlib; see the module's documentation for alternative uses
  from imp import reload


## 1. Data Preparation

### 1.1 Topic Selection

In [3]:
econ_b = pd.read_json('../datasets/เศรษฐกิจ_processed.json')
econ_m = pd.read_json('../datasets/economy_processed.json')

### 1.2 Bag-of-Words

In [4]:
# Finction to store n_word in dict
def featurize(token_list):
    token_list=token_list
    features = {}
    for token in token_list:
        features[token]=1
    return features

In [5]:
econ_b_bow = econ_b['article_tokenize'].apply(featurize)
econ_m_bow = econ_m['article_tokenize'].apply(featurize)

In [6]:
econ_b_bow.shape, econ_m_bow.shape

((6801,), (5884,))

In [7]:
vectorizer = DictVectorizer(sparse=True)
econ_b_vec = vectorizer.fit_transform(econ_b_bow)
econ_m_vec = vectorizer.fit_transform(econ_m_bow)

In [8]:
econ_b_vec, econ_m_vec

(<6801x35157 sparse matrix of type '<class 'numpy.float64'>'
 	with 844532 stored elements in Compressed Sparse Row format>,
 <5884x40835 sparse matrix of type '<class 'numpy.float64'>'
 	with 891436 stored elements in Compressed Sparse Row format>)

## 2. Model Comparison

### 2.1 K-Means

In [80]:
def kmeans_topic(data_vec=None, n_clusters=10, ):
    kmeans = KMeans(n_clusters=n_clusters, 
                    init='k-means++', 
                    max_iter=100, 
                    n_init=1, 
                    random_state=42)

    kmeans.fit(data_vec)

    order_centroids = kmeans.cluster_centers_.argsort()[:, ::-1]
    
    if data_vec == econ_m_vec:
        terms = vectorizer_1.get_feature_names_out()
    else:
        terms = vectorizer_2.get_feature_names_out()

    for i in range(n_clusters):
        print(f'Topic {i+1}')
        print('-'*10)
        for i in order_centroids[i,:10]:
            print(terms[i])
        print('\n')

In [84]:
kmeans_topic(data_vec=econ_b_vec, n_clusters=10)

Topic 1
----------
ช่างคิด
คงกระพัน
ปัตย์
ตากแห้ง
241
without
18.5
สวามิภักดิ์
ฟ็อกซ์
ราม


Topic 2
----------
0
จำยอม
น้ำเน่า
ผู้บรรยาย
onedee
ศิริลักษณ์
ยาแผนโบราณ
ปัตย์
WiFi-CAT
ตื่นตระหนก


Topic 3
----------
กลุ่มตัวอย่าง
ปรึกษาหารือ
รากฐาน
วจา
คงที่
ทางวิชาการ
08
take
ชื่อสกุล
ซึ


Topic 4
----------
ซีคอนสแควร์
ออกดอกออกผล
ดังที่
15.72
14.72
08
ปรึกษาหารือ
241
หมดอายุ
พิมาย


Topic 5
----------
ส์"
ฟ้า
กุมภาพันธ์
ฟ้อ
รายชื่อ
บางบาล
ความต้องการ
อวสาน
things
18.5


Topic 6
----------
การบรรลุผล
ตกรอบ
Utility
ช่างคิด
ผยง
Tile
องศา
พอได้
คั่ว
Muang


Topic 7
----------
สะบักสะบอม
ลงสี
ฉ้อฉล
ภาคอีสาน
กรมปศุสัตว์
อาละวาด
ศิริลักษณ์
คณะที่ปรึกษา
ฉีด
สปริงส์


Topic 8
----------
การบรรทุก
หิน
ที่ทาง
1.9
97.5
ค่าเช่า
Trainer
2025
ATTENTION
molecule


Topic 9
----------
คนใดคนหนึ่ง
ตัด
Tile
ภาวะการณ์
ตากแห้ง
ออกซิเจน
การซ่อมแซม
1,366.59
วาณิชธนกิจ
ภัสร์


Topic 10
----------
บล็อก
ออกดอกออกผล
ขนถ่ายสินค้า
1,366.59
ซีคอนสแควร์
คนใดคนหนึ่ง
การซ่อมแซม
วาณิชธนกิจ
2025
ภัสร์




### 2.2 LDA

In [9]:
# Function for modeling with LDA
def lda_model(data=None, num_topics=None): #, topicid=None
    
    dictionary = Dictionary(data)
    corpus = [dictionary.doc2bow(txt) for txt in data]
    
    model = LdaModel(corpus=corpus, num_topics=num_topics)
    #topic = pd.DataFrame(model.get_topic_terms(topicid=1, topn=20)).rename(columns={0:'index', 1:'probability'})
    for i in range(num_topics):
        top_n = [dictionary[index] for index, prob in model.get_topic_terms(topicid=i, topn=30)]
        print(f'Topic {i+1}')
        print(top_n)
        print('-'*60)
    return model

In [10]:
# Visualization
def lda_vis(data=None, num_topics=20):
    dictionary = Dictionary(data)
    corpus = [dictionary.doc2bow(txt) for txt in data]
    model = LdaModel(corpus=corpus, num_topics=num_topics)
    pyLDAvis.enable_notebook()
    viz = pyLDAvis.gensim_models.prepare(model, corpus, dictionary)
    return viz

In [11]:
lda_10b = lda_model(data=econ_b['article_tokenize'], num_topics=10)

Topic 1
['ไทย', 'บิน', 'บริษัท', 'กิจการ', 'วันที่', 'ประเทศ', 'แผน', 'ฟื้นฟู', 'บาท', '2', 'ธุรกิจ', 'จำกัด', '2563', 'โควิด', '19', 'สายการบิน', 'ปี', 'เรื่อง', '1', 'ทำ', 'เดินทาง', 'เข้ามา', 'ประกาศ', 'เครื่องบิน', 'คน', 'ล้าน', '3', 'ให้บริการ', 'โรค', 'อากาศยาน']
------------------------------------------------------------
Topic 2
['ราคา', 'น้ำมันดิบ', '1', 'สหรัฐฯ', 'ปรับตัว', 'เพิ่มขึ้น', 'บาร์เรล', 'ลดลง', 'น้ำมัน', '19', 'ล้าน', 'โควิด', 'การผลิต', 'เดือน', '63', 'ระดับ', 'ดอลลาร์', 'ประเทศ', 'ลด', 'ดูไบ', 'ความต้องการ', 'ปริมาณ', 'น้ำมันดีเซล', 'น้ำมันเบนซิน', 'ตลาด', 'วันที่', 'สัปดาห์', 'บริษัท', 'มาตรการ', 'แพร่ระบาด']
------------------------------------------------------------
Topic 3
['รายละเอียด', 'ดู', '10', 'ปัง', 'Pro', 'T', 'Mi', 'ส่อง', 'เบื้องหลัง', 'เดือน', 'หนัก', 'กรมอุตุนิยมวิทยา', 'ไทย', '30', 'จ่าย', 'ตลาด', 'ประเทศ', 'สภาพคล่อง', 'เอสเอ็มอี', 'ต่อเนื่อง', 'รู้', 'ชี้', 'ใช้ประโยชน์', 'ไท', 'มีเดีย', 'วิศวกรรม', 'โซเชียล', 'พยากรณ์อากาศ', 'ภาษี', 'ฝนตก']
-

In [14]:
import warnings
warnings.filterwarnings('ignore', category=DeprecationWarning)

In [20]:
#lda_vis(data=econ_b['article_tokenize'], num_topics=10)
# Topic overlaps

In [ ]:
lda_7b = lda_model(data=econ_b['article_tokenize'], num_topics=7)

In [ ]:
#lda_vis(data=econ_b['article_tokenize'], num_topics=7)
# Topic overlap to the low right

In [ ]:
lda_15b = lda_model(data=econ_b['article_tokenize'], num_topics=15)

In [ ]:
#lda_vis(data=econ_b['article_tokenize'], num_topics=15)
# To the left with one to the right

In [ ]:
lda_30b = lda_model(data=econ_b['article_tokenize'], num_topics=30)

In [19]:
#lda_vis(data=econ_b['article_tokenize'], num_topics=15)

In [ ]:
lda_10m = lda_model(data=econ_m['article_tokenize'], num_topics=10)

In [ ]:
#overlap กันสูง
#lda_vis(data=econ_m['article_tokenize'], num_topics=10)

In [94]:
lda_7m = lda_model(data=econ_m['article_tokenize'], num_topics=7)

Topic 1
['ไทย', '19', 'สถานการณ์', 'เศรษฐกิจ', 'ธุรกิจ', 'โควิด', 'มาตรการ', 'ประเทศ', 'ปี', '2563', 'วันที่', 'ลูกค้า', 'เรื่อง', 'ทำ', '1', 'หนี้', '2', 'ผลกระทบ', 'สำหรับ', 'ลด', 'กล่าวว่า', 'บริษัท', 'ลูกหนี้', 'หุ้น', 'พนักงาน', 'ธปท.', 'ธนาคาร', '3', 'ล้าน', 'ระดับ']
------------------------------------------------------------
Topic 2
['2', 'รถ', 'บริเวณ', 'ไทย', 'ประเทศ', 'บาท', 'วันที่', 'ราคา', 'รถยนต์', 'อุณหภูมิ', 'ระบบ', 'ฝน', '1', '3', 'องศาเซลเซียส', '2563', 'ทองคำ', 'เมตร', 'สำหรับ', 'พื้นที่', 'คลื่น', 'ปี', '4', 'ยาง', 'จังหวัด', 'รุ่น', 'เวลา', 'ประกาศ', '5', 'แรง']
------------------------------------------------------------
Topic 3
['ไทย', 'ประเทศ', 'ปี', 'วันที่', '2', 'โครงการ', 'บาท', 'ล้าน', 'คน', '1', 'ประชาชน', '2563', 'เศรษฐกิจ', 'บริษัท', '3', 'สถานการณ์', 'Matichon', 'เกาะติด', 'สำหรับ', '5', 'ข้อมูล', 'เรื่อง', 'กล่าวว่า', 'ระบบ', '19', 'เดือน', 'มาตรการ', 'จำนวน', 'รัฐบาล', 'โควิด']
------------------------------------------------------------
Topic 4
['บา

In [21]:
#lda_vis(data=econ_m['article_tokenize'], num_topics=7)

### 2.3 NMF

In [48]:
#data = econ_b['article_tokenize'].apply(lambda x:' '.join(x))
#cvec = CountVectorizer(token_pattern= "\b[A-zก-๙][A-z\.\-ก-๙]*\b")
#data = cvec.fit_transform(data)

In [49]:
nmf = NMF(n_components=10, random_state=42)
nmf.fit(econ_b_vec)

NMF(n_components=10, random_state=42)

In [50]:
nmf_features = nmf.transform(econ_b_vec)
nmf_features.shape

(6801, 10)

In [51]:
nmf.components_.shape

(10, 35157)

In [61]:
econ_b_components = pd.DataFrame(nmf.components_, columns=vectorizer_1.get_feature_names())
econ_b_components 

,0,0.0,0.00,0.001,0.003,0.005,0.0098,0.01,0.010,0.014,...,์เบิต,์เบิร์ก,์เวง,์เฮ้าส์,ํ่า,ํ้า,๒,๒๐๑๙,๒๕๔๑,๒๕๖๓
0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.013844,0.000000,0.000000,0.000491,0.000045
1,0.000000,0.000000,0.000000,0.000000,0.001582,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000809,0.000000,0.000000,0.000000,0.000000
2,0.107089,0.005027,0.002787,0.000134,0.000000,0.000134,0.156440,0.025602,0.000000,0.000000,...,0.000928,0.000000,0.000000,0.002389,0.003974,0.000000,0.000000,0.000000,0.000000,0.000000
3,0.031088,0.000000,0.000000,0.000000,0.000108,0.000000,0.000000,0.005034,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000042
4,0.051447,0.000000,0.000000,0.000278,0.000473,0.000278,0.097480,0.023438,0.000862,0.000000,...,0.000000,0.000782,0.001125,0.000000,0.000000,0.000000,0.001757,0.001757,0.000250,0.002077
5,0.055716,0.000000,0.000000,0.003978,0.000000,0.003978,0.000000,0.004295,0.003390,0.003535,...,0.000312,0.000488,0.003625,0.002073,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
6,0.012380,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.001368,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
7,0.004262,0.000691,0.012437,0.000000,0.000000,0.000000,0.063902,0.000000,0.000538,0.001078,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000510,0.000000,0.000000,0.000000,0.000000
8,0.041804,0.000000,0.000000,0.000000,0.000662,0.000000,0.014382,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000489,0.000000,0.000600,0.000220,0.000220,0.000000,0.000076
9,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.001679,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


In [60]:
# Topic & Words
for topic in range(10):
    topic = econ_b_components.iloc[topic]
    print(type(topic))
    print(f'For topic {topic+1} the words with the highest value are:')
    print(topic.nlargest(10))
    print('\n')

<class 'pandas.core.series.Series'>
For topic 0        1.000000
0.0      1.000000
0.00     1.000000
0.001    1.000000
0.003    1.000000
           ...   
ํ้า      1.013844
๒        1.000000
๒๐๑๙     1.000000
๒๕๔๑     1.000491
๒๕๖๓     1.000045
Name: 0, Length: 35157, dtype: float64 the words with the highest value are:
พัฒนา       2.926964
โครงการ     2.557163
การพัฒนา    2.390126
พื้นที่     2.142602
สร้าง       2.046783
ปี          1.981855
การลงทุน    1.735550
ส่งเสริม    1.728226
ระบบ        1.692924
เขต         1.596687
Name: 0, dtype: float64


<class 'pandas.core.series.Series'>
For topic 0        1.000000
0.0      1.000000
0.00     1.000000
0.001    1.000000
0.003    1.001582
           ...   
ํ้า      1.000809
๒        1.000000
๒๐๑๙     1.000000
๒๕๔๑     1.000000
๒๕๖๓     1.000000
Name: 1, Length: 35157, dtype: float64 the words with the highest value are:
กรมอุตุนิยมวิทยา    3.343484
พยากรณ์อากาศ        3.337378
ฝนตก                3.328116
ไท                  3.287378
หนัก  

### 2.4 Top2Vec
- fast learn, learn and deep learn

In [127]:
#print(Top2Vec.__doc__)

In [23]:
# min_count =
# ngram_vocab =
# embedding_model= (doc2vec)
# workers = cpu amount

In [24]:
econ_b_list = econ_b['article'].tolist()
#econ_b_list

In [26]:
top2vec = Top2Vec(documents=econ_b_list,
                  speed='fast-learn',
                  tokenizer=word_tokenize)

2022-11-14 15:29:54,186 - top2vec - INFO - Pre-processing documents for training
2022-11-14 15:30:17,329 - top2vec - INFO - Creating joint document/word embedding
2022-11-14 15:33:04,362 - top2vec - INFO - Creating lower dimension embedding of documents
2022-11-14 15:33:18,296 - top2vec - INFO - Finding dense areas of documents
2022-11-14 15:33:18,444 - top2vec - INFO - Finding topics


In [27]:
topic_sizes, topic_nums =top2vec.get_topic_sizes()
print(topic_sizes)
print(topic_nums)

[477 291 213 196 184 183 177 176 164 148 144 143 141 140 134 130 116 111
 107 106 106 105 104 101 100  96  96  91  90  87  82  81  80  80  73  69
  67  67  65  65  65  64  62  62  61  61  60  58  57  56  56  56  52  50
  44  44  41  40  38  38  37  36  35  35  35  34  32  32  31  29  29  29
  28  27  26  26  25  24]
[ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23
 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47
 48 49 50 51 52 53 54 55 56 57 58 59 60 61 62 63 64 65 66 67 68 69 70 71
 72 73 74 75 76 77]


In [28]:
topic_words, word_scores, topic_nums = top2vec.get_topics(18)
for words, scores, num in zip(topic_words, word_scores, topic_nums):
    print(num)
    print(f'Words: {words}')

0
Words: ['หนัก' 'พยากรณ์อากาศ' 'ไท' 'กรมอุตุนิยมวิทยา' 'ฝนตก' 'เบื้องหลัง'
 'คุณหญิง' 'Mi' 'ตาสว่าง' 'หยุดยั้ง' 'ปัง' 'Pro' 'ส่อง' 'กราบ' 'อ้อ'
 'วันนี้' 'T' 'ถึง' 'แดง' 'ยังคง' 'ต่อเนื่อง' 'ตกหนัก' 'ความ' '10' 'มาก'
 '13' 'ฉบับ' 'ฝน' 'ตู่' 'นันทิ' "'" 'ชี้' "'ลง" 'พปช' 'ชิง' 'อบจ.'
 'ประกาศ' 'GSH' 'สมุทรปราการ' 'ดา' 'นายก' 'ตรละ' 'มี' 'Diesel' 'E-mail'
 '7343' 'หนุน' 'ร.' 'สภาพคล่อง' 'ประตู']
1
Words: ['GSH' 'ค่างวด' 'ปรักปรำ' 'คอมฯ' 'ปั้น' 'จ่อ' 'หน่อย' 'ฮึ้ม' 'รายละเอียด'
 'หญิง' 'หนัก' 'นเรศ' 'ดู' 'เท็จ' 'โปรเจค' 'พยากรณ์อากาศ' 'ทท' 'เงินต้น'
 'ปัง' 'พนัก' 'ปลายข้าว' 'ผ่อนชำระ' 'ตรละ' 'หยุดยั้ง' 'วันนี้' 'ส่อง'
 'ทัน' 'ไท' 'Mi' 'ทฤษฎีใหม่' 'คุณหญิง' 'เบื้องหลัง' '>' 'ต้นตำรับ'
 'เพิ่มเติม' '1,265' "วุธ'" 'อสมท' 'ตาสว่าง' 'Pro' 'กราบ' 'ผู้ค้ำประกัน'
 'T' 'ป.ป.ช.' 'Tesla' 'ผิดคน' 'กรมอุตุนิยมวิทยา' 'RFP' 'ดูเหมือนว่า'
 'เจ้าสัว']
2
Words: ['ดูไบ' 'คงคลัง' 'น้ำมันเบนซิน' 'น้ำมันดิบ' 'ปรับตัว' 'บาร์เรล' 'โอเปค'
 'สวนทาง' 'น้ำมันดีเซล' 'เบรนท์' 'ประจำสัปดาห์' 'กดดัน' 'เทกซัส' '+/'
 'สห

In [119]:
documents, document_scores, document_ids = top2vec.search_documents_by_topic(topic_num=0, 
                                                                             num_docs=10)
for doc, score, doc_id in zip(documents, document_scores, document_ids):
    print(f'Document: {doc_id}, Score: {score}')
    print('-'*20)
    print(doc)
    print('-'*20)
    print()

Document: 6148, Score: 0.9918814897537231
--------------------
นายสุพัฒนพงษ์ พันธ์มีเชาว์ รองนายกรัฐมนตรี และรัฐมนตรีว่าการกระทรวงพลังงาน กล่าวถึงการปรับตัวขึ้นต่อเนื่องของดัชนีหุ้นในตลาดหลักทรัพย์แห่งประเทศไทยจนอยู่เหนือ 1,400 จุดว่าเป็นผลมาจากการที่นักลงทุนเชื่อมั่นเศรษฐกิจไทยมากขึ้นหลังจากที่บริษัทจัดอันดับเครดิตเรทติ้งเอสแอนด์พีได้คงอันดับเครดิตเรทติ้งของประเทศไทยอยู่ที่ระดับ BBB+ ทำให้เห็นว่ามาตรการและการดำเนินการต่างๆของเศรษฐกิจไปได้ดี
--------------------

Document: 4724, Score: 0.9913152456283569
--------------------
จีนกับโลกาภิวัตน์ 2.0 ส่องเบื้องหลังความปัง 'Mi 10T Pro' เปิดประตูประเทศ จ่าย 30 วัน เพิ่มสภาพคล่อง 'เอสเอ็มอี'
--------------------

Document: 5154, Score: 0.989698052406311
--------------------
นายสรวิศ ธานีโต อธิบดีกรมปศุสัตว์ กระทรวงเกษตรและสหกรณ์ เปิดเผยภายหลังลงนามบันทึกความร่วมมือในการพัฒนาและใช้งานแพลตฟอร์มโคเนื้อไทย (e-Catt) กับ บริษัท บีเวอร์เทค จำกัด ว่าการพัฒนาแพลตฟอร์มโคเนื้อไทย ดังกล่าวมีวัตถุประสงค์เพื่อใช้เป็นเครื่องมือในการบริหารโครงการสนับสนุนสินเ

In [30]:
# Create function that tokenize, remove stopwords and perform RegularExpression
def tokenize(txt):
  
    ''' 
    Description:
        A function that returns tokenized words with stop words and other regular expressions 
        that do not match English and Thai alphabets removed

    Parameters:
        txt = text string

    Return:
        word tokens as list
    '''

    stopwords = thai_stopwords()
    tokens = word_tokenize(txt, keep_whitespace = False)
    tokens = [token for token in tokens if token not in stopwords]
    tokens = [token for token in tokens if re.match(r'[A-Za-zก-\uDe5c]+', token)]
    return tokens

In [31]:
top2vec = Top2Vec(documents=econ_b_list,
                  speed='fast-learn',
                  tokenizer=tokenize)

2022-11-14 15:35:35,148 - top2vec - INFO - Pre-processing documents for training
2022-11-14 15:36:00,305 - top2vec - INFO - Creating joint document/word embedding
2022-11-14 15:37:07,822 - top2vec - INFO - Creating lower dimension embedding of documents
2022-11-14 15:37:12,415 - top2vec - INFO - Finding dense areas of documents
2022-11-14 15:37:12,550 - top2vec - INFO - Finding topics


In [32]:
topic_sizes, topic_nums =top2vec.get_topic_sizes()
print(topic_sizes)
print(topic_nums)

[1008  382  280  185  180  153  148  141  134  131  126  114  113  113
  106  102  102  101   97   92   89   89   88   85   85   84   82   76
   73   68   67   66   64   64   63   62   62   61   58   58   57   56
   55   55   53   48   47   47   46   45   43   43   43   42   42   42
   41   41   40   40   39   39   38   36   36   35   33   31   31   31
   30   29   27   25   25   25   24   24   23   22   20]
[ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23
 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47
 48 49 50 51 52 53 54 55 56 57 58 59 60 61 62 63 64 65 66 67 68 69 70 71
 72 73 74 75 76 77 78 79 80]


In [33]:
topic_words, word_scores, topic_nums = top2vec.get_topics(18)
for words, scores, num in zip(topic_words, word_scores, topic_nums):
    print(num)
    print(f'Words: {words}')

0
Words: ['เบื้องหลัง' 'ปัง' 'T' 'ส่อง' 'Mi' 'Pro' 'พยากรณ์อากาศ'
 'กรมอุตุนิยมวิทยา' 'ไท' 'หนัก' 'ฝนตก' 'หยุดยั้ง' 'กราบ' 'คุณหญิง'
 'ตาสว่าง' 'อ้อ' 'แดง' 'GSH' 'ตรละ' 'ชี้' 'ตกหนัก' 'ต่อเนื่อง' 'Diesel'
 'สภาพคล่อง' 'Tesla' 'โซเชียล' 'ใช้ประโยชน์' 'ตู่' 'ประตู' 'รู้' 'มีเดีย'
 'จ่าย' 'นันทิ' 'วิศวกรรม' 'ชิง' 'เอสเอ็มอี' 'ฉบับ' 'ฝน' 'พปช'
 'โลกาภิวัตน์' 'ปลาไหล' 'ค่างวด' 'อบจ.' 'Call' 'แสร์' 'คงคลัง' 'ประเทศ'
 'สมุทรปราการ' 'ดา' 'น้ำมันเบนซิน']
1
Words: ['วิกฤติ' 'เศรษฐกิจ' 'การฟื้นตัว' 'รุนแรง' 'อัดฉีด' 'หดตัว' 'ฟื้นตัว'
 'ทางเศรษฐกิจ' 'พึ่งพา' 'โลก' 'อ่อนแอ' 'ประคับประคอง' 'IMF' 'ตลาดหุ้น'
 'นักเศรษฐศาสตร์' 'ธนาคารกลาง' 'ความไม่แน่นอน' 'แย่' 'GDP' 'ภาวะถดถอย'
 'ตัวเลข' 'ล็อก' 'ว่างงาน' 'เปราะบาง' 'เฟด' 'ถดถอย' 'การคลัง' 'ลุกลาม'
 'กิจจะ' 'กำลังซื้อ' 'QE' 'ท้าทาย' 'ปัจจัย' 'ทั่วโลก' 'ผู้คน' 'มากขึ้น'
 'จีดีพี' 'เลวร้าย' 'ต้มยำกุ้ง' 'บทความ' 'เผชิญ' 'ความเสี่ยง' 'ระบาด'
 'ดาวน์' 'การท่องเที่ยว' 'กลายเป็น' 'การเงิน' 'ตลาดแรงงาน' 'กนง.'
 'ปิดกิจการ']
2
Words: ['น้ำมันดิบ' 'คงคลัง' 'เบรน

### 2.5 BERTopic